# CPV301 Practical Exam - Complete Practice Notebook

This notebook contains all the essential code patterns and examples you need for the practical exam.

## Essential Imports and Setup

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def show_images(images, titles, figsize=(15, 5)):
    """Helper function to display multiple images"""
    plt.figure(figsize=figsize)
    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(1, len(images), i+1)
        if len(img.shape) == 3:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            plt.imshow(img, cmap='gray')
        plt.title(title)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Create sample image for testing
# You can replace this with cv2.imread('your_image.jpg') for real images
sample_img = np.random.randint(0, 255, (300, 300, 3), dtype=np.uint8)
sample_gray = cv2.cvtColor(sample_img, cv2.COLOR_BGR2GRAY)

## 1. Point Operators

In [ ]:
# Histogram Equalization
equalized = cv2.equalizeHist(sample_gray)

# Contrast Stretching
stretched = cv2.normalize(sample_gray, None, 0, 255, cv2.NORM_MINMAX)

# Thresholding
ret, binary = cv2.threshold(sample_gray, 127, 255, cv2.THRESH_BINARY)
adaptive = cv2.adaptiveThreshold(sample_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
ret_otsu, otsu = cv2.threshold(sample_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

show_images([sample_gray, equalized, stretched, binary, adaptive, otsu], 
           ['Original', 'Equalized', 'Stretched', 'Binary', 'Adaptive', 'Otsu'])

## 2. Image Smoothing

In [ ]:
# Different smoothing filters
blur = cv2.blur(sample_img, (5,5))
gaussian = cv2.GaussianBlur(sample_img, (5,5), 0)
median = cv2.medianBlur(sample_img, 5)
bilateral = cv2.bilateralFilter(sample_img, 9, 75, 75)

show_images([sample_img, blur, gaussian, median, bilateral],
           ['Original', 'Averaging', 'Gaussian', 'Median', 'Bilateral'])

## 3. Edge Detection

In [ ]:
# Gradient operators - ALWAYS use CV_64F!
sobel_x = cv2.Sobel(sample_gray, cv2.CV_64F, 1, 0, ksize=5)
sobel_y = cv2.Sobel(sample_gray, cv2.CV_64F, 0, 1, ksize=5)
laplacian = cv2.Laplacian(sample_gray, cv2.CV_64F)

# Convert back to uint8
sobel_x_8u = np.uint8(np.absolute(sobel_x))
sobel_y_8u = np.uint8(np.absolute(sobel_y))
laplacian_8u = np.uint8(np.absolute(laplacian))

# Canny edge detection
blurred = cv2.GaussianBlur(sample_gray, (5,5), 0)
canny = cv2.Canny(blurred, 100, 200)

show_images([sample_gray, sobel_x_8u, sobel_y_8u, laplacian_8u, canny],
           ['Original', 'Sobel X', 'Sobel Y', 'Laplacian', 'Canny'])

## 4. Morphological Operations

In [ ]:
# Use binary image for morphological operations
kernel = np.ones((5,5), np.uint8)

erosion = cv2.erode(binary, kernel, iterations=1)
dilation = cv2.dilate(binary, kernel, iterations=1)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
gradient = cv2.morphologyEx(binary, cv2.MORPH_GRADIENT, kernel)

show_images([binary, erosion, dilation, opening, closing, gradient],
           ['Binary', 'Erosion', 'Dilation', 'Opening', 'Closing', 'Gradient'])

## 5. Geometric Transformations

In [ ]:
rows, cols = sample_gray.shape

# Translation
M_translate = np.float32([[1, 0, 50], [0, 1, 30]])
translated = cv2.warpAffine(sample_gray, M_translate, (cols, rows))

# Rotation
center = (cols//2, rows//2)
M_rotate = cv2.getRotationMatrix2D(center, 45, 1.0)
rotated = cv2.warpAffine(sample_gray, M_rotate, (cols, rows))

# Affine transformation
pts1 = np.float32([[50,50], [200,50], [50,200]])
pts2 = np.float32([[10,100], [200,50], [100,250]])
M_affine = cv2.getAffineTransform(pts1, pts2)
affine = cv2.warpAffine(sample_gray, M_affine, (cols, rows))

show_images([sample_gray, translated, rotated, affine],
           ['Original', 'Translated', 'Rotated', 'Affine'])

## 6. Color Space Conversion

In [ ]:
# Convert to different color spaces
rgb_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
hsv_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2HSV)
lab_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2LAB)
ycrcb_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2YCrCb)

# Display in RGB format for proper colors
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1), plt.imshow(rgb_img), plt.title('RGB'), plt.axis('off')
plt.subplot(2, 2, 2), plt.imshow(cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)), plt.title('HSV'), plt.axis('off')
plt.subplot(2, 2, 3), plt.imshow(cv2.cvtColor(lab_img, cv2.COLOR_LAB2RGB)), plt.title('LAB'), plt.axis('off')
plt.subplot(2, 2, 4), plt.imshow(cv2.cvtColor(ycrcb_img, cv2.COLOR_YCrCb2RGB)), plt.title('YCrCb'), plt.axis('off')
plt.tight_layout()
plt.show()

## 7. Feature Detection

In [ ]:
# Harris Corner Detection
gray_f32 = np.float32(sample_gray)
harris_corners = cv2.cornerHarris(gray_f32, 2, 3, 0.04)
harris_img = sample_img.copy()
harris_img[harris_corners > 0.01 * harris_corners.max()] = [0, 0, 255]

# FAST Corner Detection
fast = cv2.FastFeatureDetector_create()
fast_keypoints = fast.detect(sample_gray, None)
fast_img = cv2.drawKeypoints(sample_img, fast_keypoints, None, color=(255,0,0))

# SIFT Features
sift = cv2.SIFT_create()
sift_keypoints, sift_descriptors = sift.detectAndCompute(sample_gray, None)
sift_img = cv2.drawKeypoints(sample_img, sift_keypoints, None)

show_images([sample_img, harris_img, fast_img, sift_img],
           ['Original', 'Harris Corners', 'FAST Features', 'SIFT Features'])

## 8. Fourier Transform

In [ ]:
# FFT
f_transform = np.fft.fft2(sample_gray)
f_shift = np.fft.fftshift(f_transform)
magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)  # +1 to avoid log(0)

# Low-pass filter
rows, cols = sample_gray.shape
crow, ccol = rows//2, cols//2
mask = np.zeros((rows, cols), np.uint8)
cv2.circle(mask, (ccol, crow), 50, 1, -1)

# Apply mask and inverse transform
fshift_filtered = f_shift * mask
f_ishift = np.fft.ifftshift(fshift_filtered)
img_back = np.fft.ifft2(f_ishift)
img_back = np.abs(img_back)

show_images([sample_gray, magnitude_spectrum, mask*255, img_back.astype(np.uint8)],
           ['Original', 'Magnitude Spectrum', 'Low-pass Mask', 'Filtered Result'])

## 9. Complete Processing Pipeline Example

In [ ]:
def complete_processing_pipeline(img):
    """Example of a complete image processing pipeline"""
    
    # 1. Convert to grayscale if needed
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    
    # 2. Noise reduction
    denoised = cv2.GaussianBlur(gray, (5,5), 0)
    
    # 3. Enhance contrast
    enhanced = cv2.equalizeHist(denoised)
    
    # 4. Edge detection
    edges = cv2.Canny(enhanced, 50, 150)
    
    # 5. Morphological processing to clean up
    kernel = np.ones((3,3), np.uint8)
    cleaned = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    
    return gray, denoised, enhanced, edges, cleaned

# Apply pipeline
results = complete_processing_pipeline(sample_img)
titles = ['Original', 'Denoised', 'Enhanced', 'Edges', 'Cleaned']

show_images(results, titles)

## 10. White Balancing Example

In [ ]:
def gray_world_white_balance(img):
    """Gray World white balancing algorithm"""
    avg_b = np.mean(img[:, :, 0])
    avg_g = np.mean(img[:, :, 1])
    avg_r = np.mean(img[:, :, 2])
    
    avg = (avg_b + avg_g + avg_r) / 3
    
    scale_b = avg / avg_b
    scale_g = avg / avg_g
    scale_r = avg / avg_r
    
    balanced = np.zeros_like(img, dtype=np.float32)
    balanced[:, :, 0] = img[:, :, 0] * scale_b
    balanced[:, :, 1] = img[:, :, 1] * scale_g
    balanced[:, :, 2] = img[:, :, 2] * scale_r
    
    return np.clip(balanced, 0, 255).astype(np.uint8)

# Apply white balancing
balanced_img = gray_world_white_balance(sample_img)

show_images([sample_img, balanced_img], 
           ['Original', 'White Balanced'])

## Quick Debugging Helper

In [ ]:
def debug_image(img, title="Debug Image"):
    """Quick function to check image properties and display"""
    print(f"=== {title} ===")
    print(f"Shape: {img.shape}")
    print(f"Type: {img.dtype}")
    print(f"Min: {img.min()}, Max: {img.max()}")
    print(f"Mean: {img.mean():.2f}")
    
    plt.figure(figsize=(6, 4))
    if len(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Example usage
debug_image(sample_gray, "Sample Grayscale Image")